# Elements of a superintendent widget

The labelling widgets that superintendent supplies are fairly complex and have many
components, but at the top level, they are fairly modular and it is relatively easy
to make your own.

The `superintendent.Labeller` class, from which all labelling widgets inherit,
contains four elements:

1. The input widget
2. The display area
3. The progress bar / retraining widgets
4. The data queue

Only the first three are visible, and I'll just highlight them here:


In [1]:
from superintendent import MultiClassLabeller

widget = MultiClassLabeller(
    features=["data point 1", "data point 2", "data point 3"],
    options=["option 1", "option 2"],
)

widget.input_widget.layout.border = "4px solid green"
widget.feature_display.layout.border = "4px solid yellow"
widget.top_bar.layout.border = "4px solid orange"

widget

MultiClassLabeller(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.0),), layo…

The `input_widget` is in fact an argument to `superintendent.Labeller`.

`superintendent.MultiClassLabeller` does little more than pass the
`superintendent.controls.MulticlassSubmitter` widget as the input widget.

Therefore, if you want to customise the labelling process, you can do so.
In fact, this is deliberately designed like this because I wanted to develop
input widgets in separate, standalone packages separate to superintendent.

One such widget is `ipyannotations.PolygonAnnotator`.

So, to create a superintendent labelling tool, all I need to do is pass this
new input widget as an argument.

In this case, I will start by loading an instagram picture, taken
at the VDNKh in Moscow:


In [2]:
from ipyannotations import PointAnnotator
from superintendent import Labeller

input_widget = PointAnnotator(options=["pigeon"])

labeller = Labeller(
    features=["img/vdnkh.jpg"] * 2,
    input_widget=input_widget,
    display_func=input_widget.display
)

labeller

Labeller(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.0),)), Box(children=…

In [3]:
# hidden cell, see cell metadata
# use this to re-generate the
# annotations in the screenshot

import ipywidgets as widgets
import time

input_widget.canvas[0].sync_image_data = True
input_widget.canvas[1].sync_image_data = True

data = [
    {'type': 'point', 'label': 'pigeon', 'coordinates': (142, 264)},
    {'type': 'point', 'label': 'pigeon', 'coordinates': (237, 284)},
    {'type': 'point', 'label': 'pigeon', 'coordinates': (243, 374)},
    {'type': 'point', 'label': 'pigeon', 'coordinates': (374, 262)},
    {'type': 'point', 'label': 'pigeon', 'coordinates': (451, 266)},
    {'type': 'point', 'label': 'pigeon', 'coordinates': (388, 303)},
    {'type': 'point', 'label': 'pigeon', 'coordinates': (340, 377)},
    {'type': 'point', 'label': 'pigeon', 'coordinates': (518, 235)},
]

input_widget.data = data

input_widget.submit()

input_widget.data = data

input_widget.brightness_slider.value = 1.00

The data that `superintendent` stores for you is defined by the input widget, and since we are creating rich data annotations here, each label is in fact a list of point annotations:

In [4]:
labeller.new_labels

[[{'type': 'point', 'label': 'pigeon', 'coordinates': (142, 264)},
  {'type': 'point', 'label': 'pigeon', 'coordinates': (237, 284)},
  {'type': 'point', 'label': 'pigeon', 'coordinates': (243, 374)},
  {'type': 'point', 'label': 'pigeon', 'coordinates': (374, 262)},
  {'type': 'point', 'label': 'pigeon', 'coordinates': (451, 266)},
  {'type': 'point', 'label': 'pigeon', 'coordinates': (388, 303)},
  {'type': 'point', 'label': 'pigeon', 'coordinates': (340, 377)},
  {'type': 'point', 'label': 'pigeon', 'coordinates': (518, 235)}],
 None]

In [5]:
## hidden cell, not shown in nbsphinx
from PIL import Image
import ipywidgets as widgets
import io

def patch_widget_for_canvas(annotator):
    try:
        im = Image.new("RGB", annotator.canvas.size, color=(255, 255, 255))
        im2 = Image.fromarray(annotator.canvas[0].get_image_data())
        im3 = Image.fromarray(annotator.canvas[1].get_image_data())
        im.paste(im2, mask=im2)
        im.paste(im3, mask=im3)
        buffer = io.BytesIO()
        im.save(buffer, format="PNG")
        img_widg = widgets.Image(
            value=buffer.getvalue()
        )

        annotator.children = (img_widg,) + annotator.children[1:]
        annotator.layout.width = annotator.children[1].layout.width
        # close these so the state doesn't get saved:
        for canvas in annotator.canvas:
            canvas.close()
        annotator.canvas.close()
        del annotator.canvas
    except RuntimeError:
        pass


patch_widget_for_canvas(input_widget)


Let's unpack this a bit.

We've created an input widget: `ipyannotations.PolygonAnnotator`.
The only requirements on this widget is that it implements the
following 3 methods:

`on_submit`, `on_undo`, `on_skip`. These methods are used to register
superintendent's functionality as the thing that will happen when
people click on the different buttons and widgets.

Also, usually the argument to `display_func` needs to be "just" a
function, such as `IPython.display.display`. However, in this case,
the input actually happens *on* the image. It's very easy to solve
this: the input widget has a `display` method, which displays an
image inside the input widget; which is completely OK.

What this means is that we are very flexible when it comes to how we
compose our labelling widget.

The default options for single-/multi-class labelling are helpful,
but aren't helpful if we want more information about any given
data point. By simply devising our own input widget, which complies
with superintendent interfaces - `on_submit`, `on_undo`, `on_skip` -
we can still take advantage of superintendent's active learning and
distribution / orchestration functionality.
